In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

import yfinance as yf
import MetaTrader5 as mt5 #mt5 to access historcal data
import pandas as pd # for data analysis and calculation of technical indcator
import pandas_ta as ta
import numpy as np
import plotly.io as pio
import plotly.express as px # for data visualization
import plotly.graph_objects as go
import tdclient as TDClient
from datetime import datetime, time ,timezone #to specify the date time range for historical data
from IPython.display import display, Markdown, Latex # to display result in python notebook
from backtest import Backtester, get_ohlc_history, create_price_fig, evaluate_backtest

In [9]:
# Conecte mt5
mt5.initialize()
# logine mt5
login = 52185665
password = '3cAJz$AQiKwMq0'
server = 'ICMarketsSC-Demo'

mt5.login(login,password,server)

True

In [10]:
# this function retreives olhc data from mt5 account and return a data frame
def get_ohlc(symbol, timeframe, start_datetime, end_datetime):
    ohlc = mt5.copy_rates_range(symbol, timeframe, start_datetime, end_datetime)
    ohlc_df = pd.DataFrame(ohlc)
    ohlc_df['time'] = pd.to_datetime(ohlc_df['time'], unit= 's')
    return ohlc_df[['time', 'open', 'high', 'low', 'close']]

In [11]:
# 1 minut df
symbol = 'EURUSD'
pos_size = 1
timeframe = mt5.TIMEFRAME_M1
start_datetime = datetime(2025,3,13)
end_datetime = datetime.now()
ohlc_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# hour4 df
timeframe = mt5.TIMEFRAME_H4
hourly_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
# daily df
timeframe = mt5.TIMEFRAME_D1
daily_df = get_ohlc(symbol, timeframe, start_datetime, end_datetime)
daily_df
hourly_df
ohlc_df

,time,open,high,low,close
0,2025-03-13 00:00:00,1.08870,1.08870,1.08834,1.08866
1,2025-03-13 00:01:00,1.08866,1.08868,1.08860,1.08868
2,2025-03-13 00:02:00,1.08869,1.08869,1.08869,1.08869
3,2025-03-13 00:03:00,1.08869,1.08877,1.08869,1.08877
4,2025-03-13 00:04:00,1.08877,1.08888,1.08877,1.08886
...,...,...,...,...,...
70559,2025-05-21 01:18:00,1.12846,1.12862,1.12845,1.12853
70560,2025-05-21 01:19:00,1.12853,1.12854,1.12848,1.12848
70561,2025-05-21 01:20:00,1.12848,1.12848,1.12834,1.12835
70562,2025-05-21 01:21:00,1.12835,1.12835,1.12811,1.12823


In [16]:
#ohlc4
daily_df['ohlc'] = (daily_df['open'] + daily_df['high'] + daily_df['low'] + daily_df['close']) / 4
hourly_df['ohlc'] = (hourly_df['open'] + hourly_df['high'] + hourly_df['low'] + hourly_df['close']) / 4
#sma
daily_df['sma_1'] = ta.sma(close = daily_df['ohlc'], length= 1 )
daily_df['sma_14'] = ta.sma(close = daily_df['ohlc'], length= 14)
hourly_df['sma_11'] = ta.sma(close = hourly_df['ohlc'], length= 1)
hourly_df['sma_144'] = ta.sma(close = hourly_df['ohlc'], length= 14)
daily_df['high_20'] = ta.sma(close = daily_df['high'], length= 20 )
daily_df['low_20'] = ta.sma(close = daily_df['low'], length= 20)
daily_df

,time,open,high,low,close,ohlc4,ohlc,sma_1,sma_14,high_20,low_20
0,2025-03-13,1.08870,1.08973,1.08225,1.08514,1.086455,1.086455,1.086455,NaN,NaN,NaN
1,2025-03-14,1.08503,1.09128,1.08307,1.08802,1.086850,1.086850,1.086850,NaN,NaN,NaN
2,2025-03-17,1.08771,1.09297,1.08686,1.09214,1.089920,1.089920,1.089920,NaN,NaN,NaN
3,2025-03-18,1.09218,1.09547,1.08925,1.09447,1.092842,1.092842,1.092842,NaN,NaN,NaN
4,2025-03-19,1.09439,1.09461,1.08606,1.09025,1.091328,1.091328,1.091328,NaN,NaN,NaN
5,2025-03-20,1.09010,1.09176,1.08147,1.08533,1.087165,1.087165,1.087165,NaN,NaN,NaN
6,2025-03-21,1.08494,1.08613,1.07973,1.08124,1.083010,1.083010,1.083010,NaN,NaN,NaN
7,2025-03-24,1.08127,1.08584,1.07817,1.08008,1.081340,1.081340,1.081340,NaN,NaN,NaN
8,2025-03-25,1.08003,1.08302,1.07771,1.07913,1.079973,1.079973,1.079973,NaN,NaN,NaN
9,2025-03-26,1.07901,1.08028,1.07435,1.07525,1.077222,1.077222,1.077222,NaN,NaN,NaN


In [19]:
def entring_stoploss_profit(row, ohlc_df) :
    if row["sma_1"] > row['sma_14'] and row["low_x"] <= row["low_20"] :
        return "buy"
    elif row['sma_1'] < row['sma_14'] and row["high_x"] >= row["high_20"]:
        return "sell"
    # Merge daily indicators into minute data (asof = merge on closest earlier date)
merged_df = pd.merge_asof(ohlc_df, daily_df, on='time', direction='backward')
merged_df["signal"] = merged_df.apply(entring_stoploss_profit, axis=1, ohlc_df = ohlc_df)

# Tag first signal per day in original merged_df
merged_df['is_first_signal'] = (merged_df[merged_df["signal"].notna()].groupby(merged_df["time"].dt.date).cumcount() == 0)

# Set signal only on the first row per day, others = None
merged_df['signal'] = merged_df.apply(lambda row: row['signal'] if row['is_first_signal'] else None, axis=1)
merged_df[['signal', 'time']].dropna()

,signal,time
48903,sell,2025-04-30 00:00:00
52778,sell,2025-05-02 16:37:00
69042,buy,2025-05-20 00:00:00
